# 현우님의 실험 파일 기반으로 작성하였습니다.

# chunk to faiss with search

faiss db와 연결하여 인덱스 생성, 조회 및 데이터 조회를 수행해보는 예제입니다.

엘라스틱서치와의 결합을 통해 하이브리드 검색도 진행할 수 있습니다.

In [ ]:
!pip install python-dotenv elasticsearch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.0/654.0 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv

import joblib
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

import numpy as np
import requests

In [ ]:
load_dotenv()
FAISS_API_KEY = os.getenv("API_KEY")
print(FAISS_API_KEY)

nlp13_access_token_IJkdk3d2


In [ ]:
chunks = joblib.load('./final_documents.pkl')

In [ ]:
# faiss 클라이언트 설정
session = requests.Session()
url = "http://3.34.62.202:8060/"

In [ ]:
# 1. 텍스트 임베딩 모델 로드 (Hugging Face SentenceTransformer)
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')  # 예

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# 2. 텍스트 데이터를 벡터로 변환
text = "Faiss is a distributed, RESTful search engine."
vector = model.encode(text).tolist()  # 벡터를 리스트로 변환
print("vector", vector)
dimension = len(vector)
print("dimension:", dimension)

vector [-0.2365942895412445, -0.27777886390686035, -0.2769814729690552, -0.14857837557792664, -0.04466559365391731, 0.18730643391609192, -0.2007625699043274, -0.10773350298404694, 0.017776936292648315, -0.29106348752975464, 0.04522006958723068, 0.24614772200584412, 0.22713196277618408, 0.37153711915016174, -0.23451638221740723, -0.08502572029829025, 0.2329075038433075, -0.13126114010810852, 0.2857666313648224, -0.17179879546165466, -0.10931763052940369, 0.00026798248291015625, 0.07361255586147308, -0.10275545716285706, -0.4994634985923767, -0.15136003494262695, -0.17468708753585815, 0.15553052723407745, 0.16283676028251648, -0.216935932636261, 0.2636154592037201, -0.0073216017335653305, -0.31076136231422424, 0.20945972204208374, -0.06600108742713928, -0.25104308128356934, -0.054355770349502563, 0.12379428744316101, -0.19867360591888428, 0.13713009655475616, -0.08966313302516937, 0.22451025247573853, -0.1355590522289276, -0.0845823585987091, -0.08591365814208984, -0.13387718796730042, 0

In [ ]:
# 3. 전체 회사 이름 저장
all=[]
for chunk in chunks:
    all.append(chunk.metadata['company_name'])

company=list(set(all))
company

['CJ제일제당',
 '엘앤에프',
 '롯데렌탈',
 'SK하이닉스',
 '한화솔루션',
 'LG화학',
 '크래프톤',
 'SK케미칼',
 '네이버',
 '카카오뱅크']

In [ ]:
# 4. 회사별 인덱스 생성

for i in company:
    index_name = f"prod-labq-documents-multi-minilm-l12-v2-{i}1"
    response = session.post(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "algorithm" : 1, "dimension": dimension}, timeout=15)
    print(response.json())

{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-CJ제일제당1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-엘앤에프1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-롯데렌탈1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-SK하이닉스1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-한화솔루션1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-LG화학1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-크래프톤1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-SK케미칼1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-multi-minilm-l12-v2-네이버1'}
{'status': 'ok', 'message': '인덱스를 생성하였습니다', 'index': 'prod-labq-documents-mul

In [ ]:
# 5. 인덱스가 존재하는 지 확인
for i in company:
    index_name = f"prod-labq-documents-multi-minilm-l12-v2-{i}1"
    response = session.get(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, params={"index" : index_name}, timeout=15)
    print(response.json())

{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-CJ제일제당1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-엘앤에프1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-롯데렌탈1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-SK하이닉스1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-한화솔루션1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-LG화학1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-크래프톤1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-SK케미칼1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-네이버1'}
{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'pro

In [ ]:
# 전체 데이터를 저장해둔 DB
index_name = f"prod-labq-documents-multi-minilm-l12-v2-Test1"
response = session.get(f"{url}/api/index", headers={"x-api-key" : FAISS_API_KEY}, params={"index" : index_name}, timeout=15)
print(response.json())

{'status': 'ok', 'message': '해당 인덱스가 존재합니다.', 'index': 'prod-labq-documents-multi-minilm-l12-v2-Test1'}


In [ ]:
# 6. 회사별 데이터 삽입
for chunk in chunks:
    vectors = model.encode(chunk.page_content).tolist()
    company_name = chunk.metadata['company_name']
    index_name = f"prod-labq-documents-multi-minilm-l12-v2-{company_name}1"
    response = session.post(f"{url}/api/context", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "input_vector" : vectors}, timeout=15)
    print(response.json(),company_name,index_name)




{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status': 'ok', 'message': '데이터 삽입에 성공하였습니다.'} 크래프톤 prod-labq-documents-multi-minilm-l12-v2-크래프톤1
{'status':

In [ ]:
# 7. 쿼리를 회사명 기반으로 인식하여 DB연결
def find_matching_words(sentence, words):
    found_words = [word for word in words if word in sentence]
    return found_words if found_words else None

query_text = "CJ 제일제당의 2023년 3분기 매출과 2분기 영업이익을 알려주세요."
query_vector = model.encode(query_text).tolist()

check_word = find_matching_words(query_text.replace(" ",""), company)
check_word

['CJ제일제당']

In [ ]:
origin_retrieved_docs=[]

if check_word is not None:
    for i in check_word:
        comp_docs = [d for d in chunks if d.metadata['company_name'] == i]
        index_name = f"prod-labq-documents-multi-minilm-l12-v2-{i}1"
        response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 10}, timeout=15)
        faiss_results = response.json()['results']
        for item in faiss_results:
            print(comp_docs[item['document_id']])
            origin_retrieved_docs.append(comp_docs[item['document_id']].page_content)
            print("")
# ex-1과 ex-2중 하나를 선택하여 쿼리 안에 회사를 인식하지 못한 경우를 대처함
# ex-1. 오타 있을시 모든 데이터 다있는 DB 사용
else:
    index_name = f"prod-labq-documents-multi-minilm-l12-v2-Test1"
    response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 10}, timeout=15)
    faiss_results = response.json()['results']
    for item in faiss_results:
        print(chunks[item['document_id']])
        origin_retrieved_docs.append(chunks[item['document_id']].page_content)
        print("")

# ex-2. 오타 있을시 쿼리와 회사 이름 사이 유사도 기반으로 Top3의 DB 선정
"""
else:
    # 문장을 임베딩 벡터로 변환 (NumPy 배열)
    query_embedding = np.array(query_vector)
    document_embeddings = np.array(model.encode(company))

    # (1) 벡터 내적 계산
    dot_product = np.dot(document_embeddings, query_embedding)

    # (2) 벡터의 L2 노름(길이) 계산
    query_norm = np.linalg.norm(query_embedding)
    document_norms = np.linalg.norm(document_embeddings, axis=1)

    # (3) 코사인 유사도 계산
    cosine_similarities = dot_product / (query_norm * document_norms)

    # 결과 정렬 및 출력
    sorted_companys = [doc for doc, _ in sorted(zip(company, cosine_similarities), key=lambda x: x[1], reverse=True)]

    for i in sorted_companys[:3]:
        comp_docs = [d for d in chunks if d.metadata['company_name'] == i]
        index_name = f"prod-labq-documents-multi-minilm-l12-v2-{i}1"
        response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 10}, timeout=15)
        faiss_results = response.json()['results']
        for item in faiss_results:
            print(comp_docs[item['document_id']])
            origin_retrieved_docs.append(comp_docs[item['document_id']].page_content)
            print("")
"""

Output hidden; open in https://colab.research.google.com to view.

## Search With Elasticsearch(엘라스틱 서치는 사용하지 않았으므로, 실행 코드 부분을 바로 실행하시면 됩니다.)

In [ ]:
# Elasticsearch 클라이언트 설정
es = Elasticsearch("http://3.34.62.202:9200")

In [ ]:
es_index_name = "prod-labq-documents-multi-minilm-l12-v2-2"

In [ ]:
response = es.indices.delete(index=es_index_name)
print(response)

{'acknowledged': True}


In [ ]:
# Elasticsearch에 인덱스 존재하는 지 확인

if not es.indices.exists(index=es_index_name):
    response = es.indices.create(index=es_index_name)
    print(response)
    print("Elasticsearch 인덱스 생성 완료")
    response = es.indices.refresh(index=es_index_name)
    print(response)
else:
    print("Elasticsearch 인덱스 이미 존재")
response = es.indices.stats(index=es_index_name)
print(response)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'prod-labq-documents-multi-minilm-l12-v2-2'}
Elasticsearch 인덱스 생성 완료
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}}
{'_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_all': {'primaries': {'docs': {'count': 0, 'deleted': 0, 'total_size_in_bytes': 0}, 'shard_stats': {'total_count': 1}, 'store': {'size_in_bytes': 227, 'total_data_set_size_in_bytes': 227, 'reserved_in_bytes': 0}, 'indexing': {'index_total': 0, 'index_time_in_millis': 0, 'index_current': 0, 'index_failed': 0, 'delete_total': 0, 'delete_time_in_millis': 0, 'delete_current': 0, 'noop_update_total': 0, 'is_throttled': False, 'throttle_time_in_millis': 0, 'write_load': 0.0}, 'get': {'total': 0, 'time_in_millis': 0, 'exists_total': 0, 'exists_time_in_millis': 0, 'missing_total': 0, 'missing_time_in_millis': 0, 'current': 0}, 'search': {'open_contexts': 0, 'query_total': 0, 'query_time_in_millis': 0, 'query_current': 0, 'query_failure': 0, 'fetch_total':

In [ ]:
# 데이터 대량 삽입

for chunk in chunks:
    text = chunk.page_content
    metadata = chunk.metadata
    vector = model.encode(text).tolist()  # 벡터를 리스트로 변환

    # Elasticsearch에 데이터 삽입
    doc = {
        "uid": metadata['uid'],
        "nid": metadata['nid'],
        "text": text,
        "embedding": vector,
        "metadata": metadata

    }
    response = es.index(index=es_index_name, body=doc)
    print("Document indexed:", response)


KeyError: 'uid'

In [ ]:

query = {
    "query": {
        "match": {
            "text": query_text
        }
    }
}

# 검색 실행
response = es.search(index=es_index_name, body=query, size=5)

C:\Users\jilp1\AppData\Local\Temp\ipykernel_39544\2656234857.py:10: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es.search(index=es_index_name, body=query, size=5)


In [ ]:


print(response)
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")

{'took': 387, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 183, 'relation': 'eq'}, 'max_score': 10.353854, 'hits': [{'_index': 'prod-labq-documents-multi-minilm-l12-v2-2', '_id': '8T0cppQBLfL4KSeU1Ytc', '_score': 10.353854, '_ignored': ['metadata.content_types.html.keyword'], '_source': {'uid': '44276765-794d-4708-9855-a7cf34a2a669', 'nid': 2991, 'text': 'SK 증권은 4Q24 SK 하이닉스의 영업이익을 8.2 조원 (+17% QoQ)으로 예상한다.\n전통 B2C세트 수요 부진에 따른 Commodity 가격 하락에도 불구하고, HBM 등 고부\n가 중심의 확판과 레거시 출하 지양으로 우려보다 양호한 가격 (DRAM +10%,\nNAND -1%)을 예상하기 때문이다. DRAM 내 HBM 매출액 비중은 40% (+10%p\nQoQ)로 상승하며, 업황 우려가 본격화되는 시점에 높은 방어력을 확인할 것이다.', 'embedding': [-0.031807634979486465, 0.07579916715621948, -0.19842644035816193, -0.06987877190113068, -0.10593464970588684, -0.050324853509664536, -0.04604092240333557, 0.16656620800495148, -0.008283997885882854, 0.16237640380859375, -0.029495039954781532, 0.05522806942462921, 0.16236358880996704, -0.0737163424491

In [ ]:



## Faiss에서 검색한 결과에서 document_id를 이용해서 Elasticsearch에서 데이터를 가져와 nid를 통해 디코딩.

nid_values = [item['document_id']+1 for item in faiss_results]

query = {
    "query": {
        "terms": {
            "nid": nid_values
        }
    }
}

response = es.search(index=es_index_name, body=query)



In [ ]:
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")
    print(f"Metadata: {hit['_source']['metadata']}")
    print("")

ID: sj0bppQBLfL4KSeUwYY4, Score: 1.0
Content: {'uid': '6682eccd-29cf-48ef-ac4f-1a9f61bfd1ea', 'nid': 1648, 'text': '# SK케미칼', 'embedding': [-0.0370810404419899, 0.13330866396427155, -0.12111193686723709, -0.020460253581404686, 0.004756390582770109, -0.04190143570303917, 0.2754575312137604, 0.09036822617053986, 0.05623212456703186, 0.0061295670457184315, 0.12813904881477356, -0.10817084461450577, 0.12239859253168106, 0.025538666173815727, 0.016301119700074196, -0.08820291608572006, 0.02635965682566166, -0.04454326257109642, -0.09604798257350922, 0.02802097611129284, -0.06685570627450943, -0.022615717723965645, 0.14265702664852142, -0.009598059579730034, 0.061597276479005814, -0.0958332046866417, -0.06230057030916214, -0.031762655824422836, -0.1005207896232605, -0.07158495485782623, 0.02520197443664074, -0.04220166429877281, 0.047902025282382965, 0.06969311088323593, 0.02518806979060173, 0.05243467539548874, -0.002617109566926956, 0.1440429836511612, 0.039955027401447296, 0.0268464591354

In [ ]:
## hybrid 검색 - Faiss와 Elasticsearch를 조합해서 검색 (Faiss로 검색한 결과 중에서 Elasticsearch로 bm25 검색)

query_text = "sk 하이닉스의 내년 주가는 오를 전망이야?"
query_vector = model.encode(query_text).tolist()

response = session.post(f"{url}/api/context/search-by-vector", headers={"x-api-key" : FAISS_API_KEY}, json={"index" : index_name, "query_vector" : query_vector, "size" : 100}, timeout=15)

faiss_results = response.json()['results']

In [ ]:

for item in faiss_results:
    print(item)
    doc = {
        "query": {
            "match": {
                "nid": item['document_id'] + 1
            }
        }
    }
    response = es.search(index=es_index_name, body=doc)
    print(response["hits"]["hits"][0]["_source"])
    print("")

{'document_id': 1647, 'distance': 4.154514312744141}
{'uid': '6682eccd-29cf-48ef-ac4f-1a9f61bfd1ea', 'nid': 1648, 'text': '# SK케미칼', 'embedding': [-0.0370810404419899, 0.13330866396427155, -0.12111193686723709, -0.020460253581404686, 0.004756390582770109, -0.04190143570303917, 0.2754575312137604, 0.09036822617053986, 0.05623212456703186, 0.0061295670457184315, 0.12813904881477356, -0.10817084461450577, 0.12239859253168106, 0.025538666173815727, 0.016301119700074196, -0.08820291608572006, 0.02635965682566166, -0.04454326257109642, -0.09604798257350922, 0.02802097611129284, -0.06685570627450943, -0.022615717723965645, 0.14265702664852142, -0.009598059579730034, 0.061597276479005814, -0.0958332046866417, -0.06230057030916214, -0.031762655824422836, -0.1005207896232605, -0.07158495485782623, 0.02520197443664074, -0.04220166429877281, 0.047902025282382965, 0.06969311088323593, 0.02518806979060173, 0.05243467539548874, -0.002617109566926956, 0.1440429836511612, 0.039955027401447296, 0.026846

In [ ]:

nid_values = [item['document_id']+1 for item in faiss_results]

query = {
     "query": {
        "bool": {
            "filter": {
                "terms": {
                    "nid": nid_values
                }
            },
            "must": {
                "match": {
                    "text": query_text
                }
            }
        }
    },
    "size": 10,  # 최상위 도큐먼트 10개
    "sort": [
        {
            "_score": {
                "order": "desc"
            }
        }
    ]
}

response = es.search(index=es_index_name, body=query)


In [ ]:
for hit in response["hits"]["hits"]:
    print(f"ID: {hit['_id']}, Score: {hit['_score']}")
    print(f"Content: {hit['_source']}")
    print(f"Metadata: {hit['_source']['metadata']}")
    print("")

ID: lz0hppQBLfL4KSeUQ6B1, Score: 8.329401
Content: {'uid': 'cf24d3bd-2127-4742-b6f9-4011c9eaec3c', 'nid': 8277, 'text': '- - 내년 영업현금 활용 계획\n', 'embedding': [0.03073793649673462, 0.15221022069454193, -0.06348896771669388, -0.06330962479114532, -0.06879971921443939, -0.028168505057692528, 0.18609756231307983, 0.017175639048218727, -0.052078764885663986, 0.07632709294557571, 0.15546293556690216, -0.05770432949066162, 0.06667841970920563, -0.019824707880616188, 0.16403374075889587, -0.03396424651145935, 0.06971000134944916, -0.0687241330742836, 0.08839192986488342, -0.09666100889444351, 0.008988841436803341, -0.0882948487997055, 0.025869999080896378, -0.09404242783784866, 0.1485063135623932, -0.11223360151052475, 0.019833911210298538, 0.07842333614826202, 0.0292989369481802, -0.04993797466158867, 0.0807996317744255, -0.02636762335896492, 0.14887075126171112, 0.052137769758701324, 0.07461860030889511, 0.11478850990533829, 0.05204196646809578, -0.012230915948748589, -0.00693689938634634, 0.1

# 실행 코드

In [ ]:
pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.32
    Uninstalling langchain-core-0.3.32:
      Successfully uninstalled langchain-core-0.3.32


문서 재검증

In [ ]:
from langchain_openai import ChatOpenAI
import re

client = ChatOpenAI(
    model='gpt-4o-mini',
    openai_api_key='api',
    max_tokens=8000,
    temperature=0.8
)

def get_final_answer(text):
    matches = re.findall(r'최종적인 답변: True', text)
    return list(matches)

def get_false_documents(text):
    matches = re.findall(r'Document(\d+): False', text)
    return list(map(int, matches))

def remove_false_documents(doc_list, false_indices):
    return [doc for idx, doc in enumerate(doc_list, start=1) if idx not in false_indices]

def extract_answer(text):
    match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL)
    return match.group(1).strip() if match else None

def get_message(docs, question):
    docs_content = "\n".join([f"Document{i+1}: {doc}" for i, doc in enumerate(docs)])
    message = [
        {"role": "system", "content": "당신은 여러 단계(Step-by-step)로 문제를 해결해 나가는 멀티턴 접근 방식을 사용할 것입니다. 각 단계마다 reasoning(사고 과정)을 <think> ... </think> 태그 안에 작성하고, 최종 결론은 <answer> ... </answer> 태그 안에 작성하세요."},
        {"role": "user", "content": f"다음은 Question에 관련된 문서들 입니다.\n{docs_content}\n이 문서를 바탕으로 사용자의 Question을 받아서 단계별로 생각한 과정을 보여주고, 최종적인 답변을 작성해주세요.\n\nQuestion: {question}"}
    ]
    return message

def document_truth(message, response):
    message.append({"role": "assistant", "content": response})
    message.append({"role": "user", "content": "참조한 문서를 토대로 정확한 답변을 하는 것이 가능하다면 True, 불가능하다면 False를 반환하는 예시입니다."})
    message.append({"role": "assistant", "content": """
    Document1: True
    Document2: False
    Document3: True
    Document4: False
    Document5: True

    최종적인 답변: True
    """})
    message.append({'role': 'user', 'content': "이와 같은 형태로 정확한 답변을 하는 것이 가능한지 각각의 문서마다 True or False로 평가하고, 최종적인 답변이 맞는지 예시와 같은 형태로 알려주세요."})
    return message

attempt = 0
max_attempts = 3  # 반복 횟수 제한
retrieved_docs = origin_retrieved_docs # 초기 문서 리스트
filtered_docs = retrieved_docs[:5]

while attempt < max_attempts:
    attempt += 1

    # 1차 응답 생성
    message = get_message(filtered_docs, query_text)
    Response = ""
    for chunk in client.stream(message):
        Response += chunk.content
    print(Response)

    # 문서 신뢰도 평가 요청
    message = document_truth(message, Response)
    confidence = ""
    for chunk in client.stream(message):
        confidence += chunk.content
    print(confidence)

    # 만약 최종적으로 True를 반환하면 답변 출력 후 종료
    if len(get_final_answer(confidence)) == 1:
        print(extract_answer(Response))
        break

    # False로 판별된 문서 제거 후 다시 시도
    false_docs = get_false_documents(confidence)
    retrieved_docs = remove_false_documents(retrieved_docs, false_docs)
    print(f"남은 문서 개수:{len(retrieved_docs)}")

    # 문서가 모두 제거되면 반복 종료
    if not retrieved_docs:
        print("모든 문서가 신뢰할 수 없는 것으로 판단되었습니다.")
        break

# 최종적인 답변은 True-> 이렇게 반환해도 정답을 추론한 document가 True인것은 보존되기 때문에 다음 답변에서 대답가능

<think>
먼저 질문에서 요구하는 CJ제일제당의 2023년 3분기 매출과 2분기 영업이익을 찾기 위해 주어진 문서들을 살펴보아야 합니다. 

문서2에서 "3Q24P"의 매출액은 7,414.3 십억원으로 나와 있는데, 이는 2024년 3분기 예측치입니다. 하지만 그와 함께 "3Q23"라는 항목이 있어, 2023년 3분기 매출액이 7,443.4 십억원이라는 것을 확인할 수 있습니다. 

또한, 2분기 영업이익을 확인하기 위해서는 문서3을 참고해야 합니다. 이 문서에서는 2023년 2분기의 영업이익이 376 십억원으로 기재되어 있습니다.

따라서, 2023년 3분기 매출과 2분기 영업이익을 각각 확인했습니다.
</think>

<answer>
CJ제일제당의 2023년 3분기 매출은 7,443.4 십억원이며, 2023년 2분기 영업이익은 376 십억원입니다.
</answer>
<think>
문서들을 검토하여 2023년 3분기 매출과 2023년 2분기 영업이익에 대한 정확한 정보가 포함되어 있는지 확인해야 합니다. 

1. **Document1**: 이 문서에서는 2023년의 매출이나 영업이익에 대한 정보가 없으므로 False.
2. **Document2**: 여기서 2023년 3분기 매출(7,443.4 십억원)을 확인할 수 있으므로 True. 하지만 2분기 영업이익 정보는 포함되어 있지 않으므로 False.
3. **Document3**: 2023년 2분기 영업이익(376 십억원) 정보가 포함되어 있으므로 True. 그러나 3분기 매출에 대한 정보는 없다면 False.
4. **Document4**: 이 문서는 2023년의 매출 및 영업이익에 대한 정보가 없으므로 False.
5. **Document5**: 문서에서는 2023년 3분기 매출 및 2분기 영업이익 정보가 확인되지 않으므로 False.

각 문서의 평가 결과를 종합하면, 필요한 정보를 정확히 제공하는 문서는 Document2와 Document3으로 확인되었습니다.

따라서, 최종적으로 질문에 대한 정확

# deepseek api가 아직도 사용불가능 하므로 gpt 4o mini 프롬프트를 그대로 사용하였으므로, deepseek api가 사용 가능해진 이후에 수정이 필요합니다.

In [ ]:
from langchain_openai import ChatOpenAI
import re

client = ChatOpenAI(
    model='deepseek-reasoner',
    openai_api_key='api',
    openai_api_base='https://api.deepseek.com',
    max_tokens=8000,
    temperature=0.8
)

def get_final_answer(text):
    matches = re.findall(r'최종적인 답변: True', text)
    return list(matches)

def get_false_documents(text):
    matches = re.findall(r'Document(\d+): False', text)
    return list(map(int, matches))

def remove_false_documents(doc_list, false_indices):
    return [doc for idx, doc in enumerate(doc_list, start=1) if idx not in false_indices]

def extract_answer(text):
    match = re.search(r'<answer>(.*?)</answer>', text, re.DOTALL)
    return match.group(1).strip() if match else None

def get_message(docs, question):
    docs_content = "\n".join([f"Document{i+1}: {doc}" for i, doc in enumerate(docs)])
    message = [
        {"role": "system", "content": "당신은 여러 단계(Step-by-step)로 문제를 해결해 나가는 멀티턴 접근 방식을 사용할 것입니다. 각 단계마다 reasoning(사고 과정)을 <think> ... </think> 태그 안에 작성하고, 최종 결론은 <answer> ... </answer> 태그 안에 작성하세요."},
        {"role": "user", "content": f"다음은 Question에 관련된 문서들 입니다.\n{docs_content}\n이 문서를 바탕으로 사용자의 Question을 받아서 단계별로 생각한 과정을 보여주고, 최종적인 답변을 작성해주세요.\n\nQuestion: {question}"}
    ]
    return message

def document_truth(message, response):
    message.append({"role": "assistant", "content": response})
    message.append({"role": "user", "content": "참조한 문서를 토대로 정확한 답변을 하는 것이 가능하다면 True, 불가능하다면 False를 반환하는 예시입니다."})
    message.append({"role": "assistant", "content": """
    Document1: True
    Document2: False
    Document3: True
    Document4: False
    Document5: True

    최종적인 답변: True
    """})
    message.append({'role': 'user', 'content': "이와 같은 형태로 정확한 답변을 하는 것이 가능한지 각각의 문서마다 True or False로 평가하고, 최종적인 답변이 맞는지 예시와 같은 형태로 알려주세요."})
    return message

attempt = 0
max_attempts = 3  # 반복 횟수 제한
retrieved_docs = origin_retrieved_docs # 초기 문서 리스트
filtered_docs = retrieved_docs[:5]

while attempt < max_attempts:
    attempt += 1

    # 1차 응답 생성
    message = get_message(filtered_docs, query_text)
    Response = ""
    for chunk in client.stream(message):
        Response += chunk.content
    print(Response)

    # 문서 신뢰도 평가 요청
    message = document_truth(message, Response)
    confidence = ""
    for chunk in client.stream(message):
        confidence += chunk.content
    print(confidence)

    # 만약 최종적으로 True를 반환하면 답변 출력 후 종료
    if len(get_final_answer(confidence)) == 1:
        print(extract_answer(Response))
        break

    # False로 판별된 문서 제거 후 다시 시도
    false_docs = get_false_documents(confidence)
    retrieved_docs = remove_false_documents(retrieved_docs, false_docs)
    print(f"남은 문서 개수:{len(retrieved_docs)}")

    # 문서가 모두 제거되면 반복 종료
    if not retrieved_docs:
        print("모든 문서가 신뢰할 수 없는 것으로 판단되었습니다.")
        break
